In [1]:
import torch
torch.__version__


'2.4.0+cu121'

In [2]:
scalar = torch.tensor([[[1, 2], [3, 4], [5, 6]]])
print(scalar, scalar.ndim, scalar.shape)

tensor([[[1, 2],
         [3, 4],
         [5, 6]]]) 3 torch.Size([1, 3, 2])


In [3]:
scalar.device

device(type='cpu')

In [4]:
!nvidia-smi

Fri Oct 25 15:34:23 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.107.02             Driver Version: 550.107.02     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060        Off |   00000000:01:00.0  On |                  N/A |
| 32%   30C    P8             N/A /  115W |     651MiB /   8188MiB |     34%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import torch

start = 0
end = 1
step = 0.02

weight = 0.7
bias = 0.3

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

train_split = int(0.8 * len(X)) # 80% of data used for training set, 20% for testing 
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

In [6]:
model = torch.nn.Linear(1, 1)
model.weight, model.bias

(Parameter containing:
 tensor([[0.6682]], requires_grad=True),
 Parameter containing:
 tensor([-0.3584], requires_grad=True))

In [7]:
model(X) # forward (derdoor jagen)

tensor([[-0.3584],
        [-0.3450],
        [-0.3316],
        [-0.3183],
        [-0.3049],
        [-0.2915],
        [-0.2782],
        [-0.2648],
        [-0.2515],
        [-0.2381],
        [-0.2247],
        [-0.2114],
        [-0.1980],
        [-0.1846],
        [-0.1713],
        [-0.1579],
        [-0.1445],
        [-0.1312],
        [-0.1178],
        [-0.1044],
        [-0.0911],
        [-0.0777],
        [-0.0643],
        [-0.0510],
        [-0.0376],
        [-0.0243],
        [-0.0109],
        [ 0.0025],
        [ 0.0158],
        [ 0.0292],
        [ 0.0426],
        [ 0.0559],
        [ 0.0693],
        [ 0.0827],
        [ 0.0960],
        [ 0.1094],
        [ 0.1228],
        [ 0.1361],
        [ 0.1495],
        [ 0.1629],
        [ 0.1762],
        [ 0.1896],
        [ 0.2029],
        [ 0.2163],
        [ 0.2297],
        [ 0.2430],
        [ 0.2564],
        [ 0.2698],
        [ 0.2831],
        [ 0.2965]], grad_fn=<AddmmBackward0>)

In [8]:
with torch.inference_mode():
    print(model(X))

tensor([[-0.3584],
        [-0.3450],
        [-0.3316],
        [-0.3183],
        [-0.3049],
        [-0.2915],
        [-0.2782],
        [-0.2648],
        [-0.2515],
        [-0.2381],
        [-0.2247],
        [-0.2114],
        [-0.1980],
        [-0.1846],
        [-0.1713],
        [-0.1579],
        [-0.1445],
        [-0.1312],
        [-0.1178],
        [-0.1044],
        [-0.0911],
        [-0.0777],
        [-0.0643],
        [-0.0510],
        [-0.0376],
        [-0.0243],
        [-0.0109],
        [ 0.0025],
        [ 0.0158],
        [ 0.0292],
        [ 0.0426],
        [ 0.0559],
        [ 0.0693],
        [ 0.0827],
        [ 0.0960],
        [ 0.1094],
        [ 0.1228],
        [ 0.1361],
        [ 0.1495],
        [ 0.1629],
        [ 0.1762],
        [ 0.1896],
        [ 0.2029],
        [ 0.2163],
        [ 0.2297],
        [ 0.2430],
        [ 0.2564],
        [ 0.2698],
        [ 0.2831],
        [ 0.2965]])


In [9]:
loss_fn = torch.nn.L1Loss() # MAE loss is same as L1Loss
optimizer = torch.optim.SGD(params=model.parameters(), # parameters of target model to optimize
                            lr=0.01) # learning rate (how much the optimizer should change parameters at each step, higher=more (less stable), lower=less (might take a long time))

In [10]:
torch.manual_seed(42)

# Set the number of epochs (how many times the model will pass over the training data)
epochs = 100

# Create empty loss lists to track values
train_loss_values = []
test_loss_values = []
epoch_count = []

for epoch in range(epochs):
    ### Training

    # Put model in training mode (this is the default state of a model)
    model.train()

    # 1. Forward pass on train data using the forward() method inside 
    y_pred = model(X_train)
    # print(y_pred)

    # 2. Calculate the loss (how different are our models predictions to the ground truth)
    loss = loss_fn(y_pred, y_train)

    # 3. Zero grad of the optimizer
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Progress the optimizer
    optimizer.step()

    ### Testing

    # Put the model in evaluation mode
    model.eval()

    with torch.inference_mode():
      # 1. Forward pass on test data
      test_pred = model(X_test)

      # 2. Caculate loss on test data
      test_loss = loss_fn(test_pred, y_test.type(torch.float)) # predictions come in torch.float datatype, so comparisons need to be done with tensors of the same type

      # Print out what's happening
      if epoch % 10 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.detach().cpu().numpy())
            test_loss_values.append(test_loss.detach().cpu().numpy())
            print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss} ")

Epoch: 0 | MAE Train Loss: 0.6707576513290405 | MAE Test Loss: 0.6731709241867065 
Epoch: 10 | MAE Train Loss: 0.555547833442688 | MAE Test Loss: 0.5384610295295715 
Epoch: 20 | MAE Train Loss: 0.4403378367424011 | MAE Test Loss: 0.40375107526779175 
Epoch: 30 | MAE Train Loss: 0.32512789964675903 | MAE Test Loss: 0.2690412104129791 
Epoch: 40 | MAE Train Loss: 0.20991790294647217 | MAE Test Loss: 0.13433125615119934 
Epoch: 50 | MAE Train Loss: 0.09470794349908829 | MAE Test Loss: 0.008356565609574318 
Epoch: 60 | MAE Train Loss: 0.03896244615316391 | MAE Test Loss: 0.07178118079900742 
Epoch: 70 | MAE Train Loss: 0.033962421119213104 | MAE Test Loss: 0.07603313028812408 
Epoch: 80 | MAE Train Loss: 0.03046739101409912 | MAE Test Loss: 0.07050827145576477 
Epoch: 90 | MAE Train Loss: 0.02702857181429863 | MAE Test Loss: 0.062235765159130096 


tensor(0.0551)